# SQL, Spark, Tables, Reporting ...

* SQL Abfragen auf Dataframes anwenden
* Mit Spark Datenbanken & Tabellen arbeiten
* User-Defined Functions benutzen
* Opertionen auf mehrere Datenmegen gleichzeitig anwenden
* Kleiner Deep Dive auf Windmow Operationen
* Testdaten erzeugen die zwei Geschäftsprozesse aus der Versicherungswelt abbilden
* Die ersten (und wesentlichsten) Schritte für ein sinnvolles Monitoring dieser Prozesse gehen


## Wie immer eine Spark-Application registieren

In [39]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
        .builder
        .appName("sql")
        .config("spark.dynamicAllocation.enabled",False)
        .config("spark.sql.adaptive.enabled",False)
        .master("local[4]")
        .getOrCreate()
)
spark

## Nun Testdaten erzeugen
Hierzu haben wir ein Beispiel aus der Versicherungswelt vorbereiten. Vielleicht ist es dem in der Signal sogar ein bisschen ähnlich ;-)


![](architecture-level-01.dio.svg)

In [41]:
import generate_test_data
generate_test_data.main(1000)

Wir sehen nun auf im Dateibrowser eine Liste von Dateien, die überwiegend fachliche Events enthalten, die zwei Geschäftsprozesse abbilden:

### Ein Kunden bekommt einen neuen Vertrag

Die Systeme welche die Events schreiben liegen in der Verantwortung verschiedener Teams:

#### Team Antrag
 * **Antrag Erzeugt:** Der Kunde geht im Internet auf ein System um einen Antrag auf einen Versicherungsvertrag zu stellen. Dieses System validert schon ein wenig, versucht einen schon bestehenden Kunden im Partner-Systeme zu finden und schmeißt im Anschluss dieses Business-Event.
 * **AB-Test:** das Team Antrag ist stehst bemüht die User-Experience zu optimieren. So will es zum Beispiel die Durchlaufzeit auf ihrem eigenen System zu verkürzen. So variert es z.B. die Farbe des Knopfes. Wenn es bei einem Antrag von der Standardfarbe des Knopfes abweicht, schreibt es dieses technische Event
 * **Kunde hat Angebot aktzeptiert:** Nch
 * **Kunde hat Angebot abgelehnt:**
 
#### Team Vertrag
 * **Antrag abgelehnt**: Ein Vertrag kann vom Vertragssystem abgelehnt werden. Dieses fragt z.B. noch bei Fraud an und validert auch sonst den Antrag viel "besser" als es das Antragssystem könnte.
 * **Kunde angelegt**: (eigentlich müsste ein anderes System dieses Event schreiben, aber in unserem Beispiel soll das mal OK sein) Wenn in einem Antrag nicht schon eine Referenz auf einen Kunden vorhanden ist, dann legt das Vertragssystem einen Kunden an und informiert Gott und die Welt hierüber.
 * **Vertrag angeboten**: Wenn ein Vertragsantrag aus Sicht des Vertragssystem gut aussieht, dann gibt es den Vertrag frei zur Annahme durch den Kunden. 
 * **Vertrag policiert**: Wenn der Kunden ein Vertragsangebot akzeptiert hat, policieren wir den Vertrag. Nun ist der Kunde versichert. Jetzt können bedenkenlos Schadensfälle eintreten.
 
#### Team Schaden
 * **Schaden reguliert**: wenn ein Schadensfall geprüft wurde und alles passt, dann überweisen wir Geld

#### Übrigens
**kunden.csv** ist nur ein Container mit allen Kunden ;-)



## Erste Blicke in unsere Testdaten

In [42]:
kunden_df = spark.read.option("header", True).option("inferSchema", True).csv("kunden.csv")
kunden_df.printSchema()
kunden_df.show(4, truncate=False)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)

+------------------------------------+-------------------+
|id                                  |name               |
+------------------------------------+-------------------+
|ce4cf720-4a1a-11ee-8349-7b31dfcc7252|gorzala            |
|d55a2ede-4a1a-11ee-8cce-b739d8c6e256|Scharrenberch      |
|bcecc0dc-af06-4910-b22e-12030333ab5b|Agitated Williamson|
|d19506f3-4558-4b3c-9e01-938700866ae9|Crazy Villani      |
+------------------------------------+-------------------+
only showing top 4 rows



Soweit so bekannt. Wir könnten jetzt auf die Daten zugreifen in dem wir wie in den vorhergegangen Kapiteln filter anwenden, Null-Werte entfernen usw.
Wir möchten aber absofort ganz normal mit SQL auf unsere Daten zugreifen. Dazu müssen wir zu einem Dataframe in Spark einen _View_ anlegen. Diese Views werden dann ganz normal wie Datenbanktabellen in SQL-Datenbanken behandelt (read-only).

In [43]:
kunden_df.createOrReplaceTempView("kunden")

**Beachte:** die Funtion gibt nichts zurück. In der Spark-Application ist jetzt eine eine Tabelle unter dem Namen "kunden" verfügbar!

In [44]:
spark.sql("SELECT * FROM kunden")

DataFrame[id: string, name: string]

Wir sehen das wir ein Dataframe zurückbekommen. Auf diesem können wir dann wieder die bekannten Operationen ausführen. Beachte, ob Du die Dateframe-API oder die SQL-API nutzt, hat in der Regel **keinen** Impact auf die Performance!

**Übung** setzte hier mal ein paar SQL gegen diese Tabelle ab. Filter nach bestimmten Namen... 

In [45]:
spark.sql('SELECT * FROM kunden').show()

+--------------------+--------------------+
|                  id|                name|
+--------------------+--------------------+
|ce4cf720-4a1a-11e...|             gorzala|
|d55a2ede-4a1a-11e...|       Scharrenberch|
|bcecc0dc-af06-491...| Agitated Williamson|
|d19506f3-4558-4b3...|       Crazy Villani|
|0ee2f173-8860-452...|Compassionate Noe...|
|2cf46a78-d800-48e...|       Busy Sinoussi|
|292e6eda-4dab-4f7...|    Wizardly Wozniak|
|4aa7e4ad-bb89-4c4...| Wonderful Mendeleev|
|14272797-7ddf-451...|      Great Brattain|
|5a80abe6-594e-4a0...|    Strange Roentgen|
|93cfdc26-9feb-4c6...|     Determined Borg|
|7049985c-95d9-430...|    Gracious Wescoff|
|f1573be3-8c18-4c7...|         Brave Morse|
|4e5449a8-4fed-48b...|      Elegant Agnesi|
|10107419-c873-41a...|   Beautiful Vaughan|
|0c25d304-df3a-45c...|       Elegant Kilby|
|1a0d4987-7faf-47c...|  Affectionate Gates|
|90127df4-9eda-47f...|      Zealous Pascal|
|2a45e83f-f90e-42b...|          Bold Gauss|
|89370b2b-23b7-4f2...|      Roma

## Alle Testdaten importieren
und views anlegen

In [46]:
# AB-test
ab_test_schema = "AntragsId STRING"
ab_test_df = spark.read.option("header", True).schema(ab_test_schema).csv("ab_test.csv")
ab_test_df.createOrReplaceTempView("ab_test")

# Antrag Abgelehnt
antrag_abgelehnt_schema = "AntragsId STRING, KundenId STRING, TimeStamp TIMESTAMP, Grund STRING"
antrag_abgelehnt_df = spark.read.option("header", True).schema(antrag_abgelehnt_schema).csv("antrag_abgelehnt.csv")
antrag_abgelehnt_df.createOrReplaceTempView("antrag_abgelehnt")

# Antrag Erzeugt
antrag_erzeugt_schema = "AntragsId STRING, StartZeit TIMESTAMP, EndZeit TIMESTAMP, KundenId STRING"
antrag_erzeugt_df = spark.read.option("header", True).schema(antrag_erzeugt_schema).csv("antrag_erzeugt.csv")
antrag_erzeugt_df.createOrReplaceTempView("antrag_erzeugt")

# Kunde Angelegt
kunde_angelegt_schema = "KundenId STRING, TimeStamp TIMESTAMP"
kunde_angelegt_df = spark.read.option("header", True).schema(kunde_angelegt_schema).csv("kunde_angelegt.csv")
kunde_angelegt_df.createOrReplaceTempView("kunde_angelegt")

# Kunde hat Angebot Abgelehnt
kunde_hat_angebot_abgelehnt_schema = "VertragsId STRING, AntragsId STRING, Grund STRING, TimeStamp TIMESTAMP"
kunde_hat_angebot_abgelehnt_df = spark.read.option("header", True).schema(kunde_hat_angebot_abgelehnt_schema).csv("kunde_hat_angebot_abgelehnt.csv")
kunde_hat_angebot_abgelehnt_df .createOrReplaceTempView("kunde_hat_angebot_abgelehnt")

# Kunde hat Angebot Akzeptiert
kunde_hat_angebot_akzeptiert_schema = "VertragsId STRING, AntragsId STRING, KundenId STRING, TimeStamp TIMESTAMP"
kunde_hat_angebot_akzeptiert_df = spark.read.option("header", True).schema(kunde_hat_angebot_akzeptiert_schema).csv("kunde_hat_angebot_akzeptiert.csv")
kunde_hat_angebot_akzeptiert_df.createOrReplaceTempView("kunde_hat_angebot_akzeptiert")

# Schaden Gemeldet
schaden_gemeldet_schema = "VertagsId STRING, SchadensId STRING, Schadenshoehe INT, TimeStamp TIMESTAMP"
schaden_gemeldet_df = spark.read.option("header", True).schema(schaden_gemeldet_schema).csv("schaden_gemeldet.csv")
schaden_gemeldet_df.createOrReplaceTempView("schaden_gemeldet")

# Schaden Reguliert
schaden_reguliert_schema = "SchadensId STRING, TimeStamp TIMESTAMP"
schaden_reguliert_df = spark.read.option("header", True).schema(schaden_reguliert_schema).csv("schaden_reguliert.csv")
schaden_reguliert_df.createOrReplaceTempView("schaden_reguliert")

# Vertrag Angeboten
vertrag_angeboten_schema = "VertragsId STRING, AntragsId STRING, KundenId STRING, TimeStamp TIMESTAMP"
vertrag_angeboten_df = spark.read.option("header", True).schema(vertrag_angeboten_schema).csv("vertrag_angeboten.csv")
vertrag_angeboten_df.createOrReplaceTempView("vertrag_angeboten")

# Vertrag Policiert
vertrag_policiert_schema = "VertragsId STRING, TimeStamp TIMESTAMP"
vertrag_policiert_df = spark.read.option("header", True).schema(vertrag_policiert_schema).csv("vertrag_policiert.csv")
vertrag_policiert_df.createOrReplaceTempView("vertrag_policiert")

In [48]:
# alles da?
spark.sql("SHOW TABLES").show(truncate=False)

+---------+----------------------------+-----------+
|namespace|tableName                   |isTemporary|
+---------+----------------------------+-----------+
|         |ab_test                     |true       |
|         |antrag_abgelehnt            |true       |
|         |antrag_erzeugt              |true       |
|         |kunde_angelegt              |true       |
|         |kunde_hat_angebot_abgelehnt |true       |
|         |kunde_hat_angebot_akzeptiert|true       |
|         |kunden                      |true       |
|         |schaden_gemeldet            |true       |
|         |schaden_reguliert           |true       |
|         |vertrag_angeboten           |true       |
|         |vertrag_policiert           |true       |
+---------+----------------------------+-----------+



In [51]:
# Stichprobe bzgl. der Datentypen?
spark.sql("DESCRIBE schaden_gemeldet").show()

+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|    VertagsId|   string|   null|
|   SchadensId|   string|   null|
|Schadenshoehe|      int|   null|
|    TimeStamp|timestamp|   null|
+-------------+---------+-------+



## Ein erstes Reporting

In [60]:
# Zeige alle Anträge an, bei den wir die Farbe des Knopfs verändert haben

# erstmal alle Anträge

spark.sql("SELECT count(1) FROM antrag_erzeugt").show()
spark.sql("""
    SELECT *
      FROM antrag_erzeugt
 INNER JOIN ab_test
        ON antrag_erzeugt.AntragsId = ab_test.AntragsId
""").show()

+--------+
|count(1)|
+--------+
|    3000|
+--------+

+--------------------+-------------------+-------------------+--------------------+--------------------+
|           AntragsId|          StartZeit|            EndZeit|            KundenId|           AntragsId|
+--------------------+-------------------+-------------------+--------------------+--------------------+
|5951a2fe-9298-44f...|2023-09-12 03:24:24|2023-09-12 03:28:24|292e6eda-4dab-4f7...|5951a2fe-9298-44f...|
|97fc3fac-8a15-419...|2023-09-11 00:28:38|2023-09-11 00:29:38|                null|97fc3fac-8a15-419...|
|fd99d05e-87f6-4a9...|2023-09-16 20:32:41|2023-09-16 20:35:41|                null|fd99d05e-87f6-4a9...|
|afc546ba-0430-4af...|2023-09-02 21:47:10|2023-09-02 21:49:10|6acf9d4f-fe9b-4be...|afc546ba-0430-4af...|
|8f942606-be75-475...|2023-09-03 14:09:28|2023-09-03 14:11:28|7e2105ed-8c7b-46b...|8f942606-be75-475...|
|4fbbbee7-fff5-4ae...|2023-09-13 04:35:39|2023-09-13 04:36:39|b2f8a2e6-8e0d-4e4...|4fbbbee7-fff5-4ae...|